In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as web
import ta

In [2]:
import matplotlib.pyplot as plt

In [3]:
cd ~/Documents/WQU/WQU_650-ml_for_finance/intro_to_deep_learning/data/

/home/neelkanth/Documents/WQU/WQU_650-ml_for_finance/intro_to_deep_learning/data


In [4]:
dataset = pd.read_csv('dataset.csv', index_col=0)

In [81]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52200 entries, 2009-09-28 09:46:35 to 2018-10-31 18:25:11
Data columns (total 6 columns):
price    52200 non-null float64
bid      52200 non-null float64
ask      52200 non-null float64
size     52200 non-null int64
v        52200 non-null int64
dv       52200 non-null int64
dtypes: float64(3), int64(3)
memory usage: 5.3 MB


In [82]:
dataset['dv'] = dataset['dv'].astype(np.int64)

In [6]:
# Converting index to datetime index
dataset.index = pd.to_datetime(dataset.index)

# Deriving 10 Min intervaled ohlc ask data
data_ask = dataset['ask'].resample('10Min').ohlc().fillna(method='ffill')

# Deriving 10 Min intervaled ohlc bid data
data_bid = dataset['bid'].resample('10Min').ohlc().fillna(method='ffill')

# Mergina both dataframes
df = data_ask.merge(data_bid, how='outer', left_index=True, right_index=True).dropna(how='all')
df.columns = ['open_ask','high_ask','low_ask','close_ask','open_bid','high_bid','low_bid','close_bid']

In [67]:
# Same treatment to price data
data_price = dataset['price'].resample('10Min').ohlc().fillna(method='ffill').dropna(how='all')

In [84]:
sp_df = data_price.merge(dataset[['v', 'dv']],how='outer', left_index=True, right_index=True).fillna(method='ffill').dropna(how='all')

# setting first values to 0
sp_df['v'][0] = 0
sp_df['dv'][0] = 0

In [92]:
# generating Awsome Oscilator`
ao = ta.momentum.ao(high=sp_df['high'],low=sp_df['low'],fillna=True)

In [93]:
# Generating relative strength index
rsi = ta.momentum.rsi(close=sp_df['close'],fillna = True)

In [94]:
# True Strength Index
tsi = ta.momentum.tsi(close=sp_df['close'], fillna=True)

In [95]:
MACD = ta.trend.macd(close=sp_df['close'], fillna=True)

In [96]:
# Williams %R  is a momentum indicator 
R_pct = ta.momentum.wr(high=sp_df['high'], low=sp_df['low'], close=sp_df['close'],fillna=True)

In [99]:
# Generating East of Movement feature
EoM = ta.volume.ease_of_movement(high=sp_df['high'], low=sp_df['low'], close=sp_df['close'], volume=sp_df['v'], fillna=True)

In [102]:
# Generating Chaikin Money Flow
CMF = ta.volume.chaikin_money_flow(high=sp_df['high'], low=sp_df['low'], close=sp_df['close'], volume=sp_df['v'], fillna=True)

In [107]:
# Generating Force Index
force_index = ta.volume.force_index(close=sp_df['close'],volume=sp_df['v'], fillna=True)